# Continuous Requirements Generation with GPT-4
This notebook explores the use of utilizing GPT 4 on a domain-specific set of requirements. This model will be used downstream to manage requirements changes provided by external sources.

### Setting Up

In [488]:
# %pip install pymupdf4llm

In [489]:
# Find next iteration increment for saving the output response
import os

# change the prefix based on the user story provided to the prompt
file_prefix = "outputs/4-powerpoint-report/iteration-"
file_extension = ".md"

i = 1
while os.path.exists(f"{file_prefix}{i}{file_extension}"):
    i += 1

output_filename = f"{file_prefix}{i}{file_extension}"

In [490]:
# Constants
MODEL_NAME = "gpt-4o"
IEEE_830 = "inputs/IEEE 830-1998.pdf"
BRIDGE_INSPECTION = "inputs/a-state-of-the-art-review-of-bridge-inspection-planning-current-situation-and-future-needs.pdf"
# SRS_PDF_PATH = "inputs/2005 - pontis.pdf"
# SRS_MD_PATH = "inputs/output_srs.md"
EDIT_MD_PATH = "inputs/edited_srs.md"
FINAL_OUTPUT_PATH = output_filename

JIRA_ISSUE = """
As an advanced user, \
I want to export bridge inspection results to a PowerPoint presentation, \
so that I can present risk analysis and recommendations to management personnel in a digestible format.
"""

# RAG_DOCUMENTS = [SRS_PDF_PATH, SRS_TXT_PATH, IEEE_830, BRIDGE_INSPECTION]
RAG_DOCUMENTS = [EDIT_MD_PATH, IEEE_830, BRIDGE_INSPECTION]

In [491]:
# Get the OpenAI api key
import json

config_data = json.load(open("config.json"))
openai_api_key = config_data["OPENAI_API_KEY"]

In [492]:
# Create the OpenAI Client
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

In [493]:
# Read the system instructions from the prompt file
with open("prompts/system_instructions.txt", "r") as file:
    system_instructions = file.read()

# Creating a Custom Assistant with File Search Capabilities

Create a new assistant with access to the existing system requirements

In [494]:
assistant = client.beta.assistants.create(
    name="Continuous Requirements Generator",
    instructions=system_instructions,
    model=MODEL_NAME,
    tools=[{"type": "file_search"}]
)

Upload the file and add them to a Vector Store

In [495]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Gemini Software System Requirements")
 
# Ready the files for upload to OpenAI
file_paths = RAG_DOCUMENTS
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)


Update the assistand to use the new Vector Store

In [496]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Create a thread

In [497]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open(EDIT_MD_PATH, "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": JIRA_ISSUE,
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_oK6iRabkrOLxViX5opp5HlX2'])


Create a run and check the output

In [498]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Based on the user story provided, here's how we can generate the related use cases, functional requirements, and non-functional requirements:

### JSON Response

```json
{
    "use-cases": [
        {
            "parent-section": "3.6 DATA MANAGEMENT",
            "id": "UC-17",
            "modification-type": "new",
            "primary-actor": "Advanced user",
            "scope": "Export bridge inspection data",
            "stakeholders": "Inspector, Routine and Advanced User",
            "precondition": "User is logged into Pontis and has the necessary permissions.",
            "description": "The user exports bridge inspection results to a PowerPoint presentation.",
            "success-end-condition": "The bridge inspection results are successfully exported to a PowerPoint presentation."
        }
    ],
    "functional-requirements": [
        {
            "parent-id": "UC-17",
            "id": "FR-17.1",
            "modification-type": "new",
            "description": 

In [499]:
# Document the inputs into this set of generated requirements
import pathlib

# Add a placeholder for observations
final_output = "# Observations\nPlaceholder for developer observations...\n\n"

# Capture the list of documents shared for RAG
document_input_header = "# Configuration"
document_list = ""
for document in RAG_DOCUMENTS:
    document_list += f"{document}\n"
final_output += f"{document_input_header}\n## RAG Files:\n{document_list}"
final_output += f"## Model Name\n{MODEL_NAME}\n"
final_output += f"## Prompt{JIRA_ISSUE}"

# Capture the system instructions prompt
system_instructions_header = "# System Instructions"
final_output += f"\n{system_instructions_header}\n{system_instructions}\n"

# Capture the final output message
llm_output_header = "# Final Output Message"
final_output += f"\n{llm_output_header}\n{message_content.value}\n"

# Save the final message to a file
pathlib.Path(FINAL_OUTPUT_PATH).write_bytes(final_output.encode())

11769